In this lab we are going to build an Azure MLS pipeline that will automatically train and then register the model.  

You should already have a 
 * train.py
 * driver_training.py
 * parameters.json 

in an experiment folder. These resources will be used as the first step in the Machine Learning pipeline created and run later in this notebook.

We also need to create a python file to register the model.  

Then we'll put it all together in one AMLS pipeline.

If you don't want to use AMLS pipelines we can just call these python files directly from our DevOps tool of choice.  

In [10]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/davew202105/code/git/MLOps-E2E-sdkv2/Lab12


In [22]:
# Set the folder for the experiment files used in previous lab
training_folder = 'driver-training'
experiment = 'driver-training'

## register_model.py
This script loads the model from where it was saved, and then registers it in the workspace. This will be the second step in the pipeline. The script is written to the experiment folder from this notebook for convenience.

Note:  if we decide not to use AMLS pipelines and just want to use a more DevOps-centric approach (using gh actions or azdo pipelines instead of AMLS pipelines), we can.  

In [23]:
%%writefile $training_folder/register_model.py
# Import libraries
import argparse
import joblib
from azureml.core import Workspace, Model, Run

# Get parameters
parser = argparse.ArgumentParser()
parser.add_argument('--model_folder', type=str, dest='model_folder', default="driver_model", help='model location')
args = parser.parse_args()
model_folder = args.model_folder

# Get the experiment run context
run = Run.get_context()

# load the model
print("Loading model from " + model_folder)
model_name = 'driver_model'
model_file = model_folder + "/" + model_name + ".pkl"

# Get metrics for registration
## TODO
## HINT: Try storing the metrics in the parent run, which will be
##       accessible during both the training and registration
##       child runs using the 'run.parent' API.
## See https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py#parent

Model.register(workspace=run.experiment.workspace,
               model_path = model_file,
               model_name = 'driver_model',
               tags={'Training context':'Pipeline'})

run.complete()

Overwriting driver-training/register_model.py


## Create an Azure Machine Learning Pipeline to Run the Scripts as a Pipeline

See [this tutorial](https://github.com/MicrosoftDocs/mslearn-aml-labs/blob/master/05-Creating_a_Pipeline.ipynb) for a starting point

Use the scikit-learn and lightgbm conda packages

In [24]:
import azureml.core
from azureml.core import Workspace

# Load the workspace
ws = Workspace.from_config()

In [25]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "automl"  #changeme

# Verify that the compute cluster exists
try:
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           vm_priority='lowpriority', 
                                                           max_nodes=4)
    pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

pipeline_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [45]:
from azureml.core import Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.core.runconfig import RunConfiguration, DockerConfiguration
from azureml.widgets import RunDetails

In [ ]:
# we need an environment which we already created and persisted in the last lab

# here's how we can see a list of our custom environments
envs = Environment.list(workspace=ws)

for env in envs:
    if not (env.startswith("AzureML")):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())
        
# the env should be called driver_training

In [30]:
env_name = experiment

# connect to the existing python env we already registered
registered_env = Environment.get(
    workspace=ws,
    name=env_name)

In [54]:
# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Assign the target of the runconfig object to the cluster created above  
pipeline_run_config.target = pipeline_cluster

# Assign the environment of the runconfig object to the registered environment
pipeline_run_config.environment = registered_env

print ("Run configuration created.")

Run configuration created.


Let's start build the pipeline in code.  For now we just need the training script and the model registration script.  We'll add more later.  

In [59]:
# Create a PipelineData (temporary Data Reference) for the model folder
model_folder = PipelineData(
    "outputs", 
    datastore=ws.get_default_datastore())

# before we created a src (ScriptRunConfig).  This is slightly different:  we create a PythonScriptStep
args = ['--output_folder',"outputs"]
docker_config = DockerConfiguration(use_docker=True)

# now we want to use that src as Step1 of the pipeline
train_step = PythonScriptStep(
        name = "Train Model",
        source_directory = training_folder,
        script_name = "driver_training.py",
        arguments=args,
        outputs=[model_folder],
        compute_target=pipeline_cluster,  # we built this above
        allow_reuse = True,
        runconfig = pipeline_run_config
    )
register_step = PythonScriptStep(
        name = "Register Model",
        source_directory = training_folder,
        script_name = "register_model.py",
        arguments=['--model_folder', model_folder],
        inputs = [model_folder],
        compute_target=pipeline_cluster,  # we built this above
        allow_reuse = True,
        runconfig = pipeline_run_config
    )
print("Pipeline steps defined")

Pipeline steps defined


In [60]:
# Construct the pipeline, which contains Step 1 & 2
pipeline_steps = [train_step, register_step]
pipeline = Pipeline(workspace = ws, steps=pipeline_steps)
print("Pipeline is built.")

Pipeline is built.


Some notes:

* **If** you have a multi-stage pipeline with separate train and register model stages, and need so log metrics on the first stage and access them from the second, please see [Azure ML Python SDK - Run Class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py) and in particular `run.parent.log()` and `run.parent.get_metrics()`. This (obviously) will only work when there *is* a parent run, i.e., doesn't work if runnning `train.py` or `register_model.py` by themselves.
* When registering models, also keep in mind the differences between `run.register_model()` and `Model.register()` from [Azure ML Python SDK - Model class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model(class)?view=azure-ml-py). Only the first allows you keep full traceability between the context of the run/dataset and the trained model. To use it, remember you must first explicitly upload the model with `run.upload_file()`, and then use the name of the model as the `model_path` parameter.
* If you deploy your model to Azure Container Instances and it just hangs with no apparent error message, check the logs for the container in Azure Container Instances (or start the container  locally using Docker and use its logs to debug the problem). **If you see this, there might possibly be an error in the `init()` function of your scoring script.**. You can try to run the container locally in your PC to diagnose this problem.

In [61]:
# Create an experiment and run the pipeline
experiment = Experiment(workspace = ws, name = 'driver-training-pipeline-v2')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)
print("Pipeline submitted for execution.")

Created step Train Model [253c6409][a5bdd85e-f60b-471b-901d-f4cd03e09740], (This step will run and generate new outputs)
Created step Register Model [69f042e0][c1631a56-c323-411b-9880-5bfb2317e400], (This step will run and generate new outputs)
Submitted PipelineRun e104aaf3-0c5a-4d82-9d94-95eb7f6b468e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e104aaf3-0c5a-4d82-9d94-95eb7f6b468e?wsid=/subscriptions/52061d21-01dd-4f9e-aca9-60fff4d67ee2/resourcegroups/MLOpsWorkshop/workspaces/mlops&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
Pipeline submitted for execution.


In [62]:
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

# try monitoring this from the Azure portal as well.  
# make sure you understand approximately what this is doing

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

PipelineRunId: e104aaf3-0c5a-4d82-9d94-95eb7f6b468e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/e104aaf3-0c5a-4d82-9d94-95eb7f6b468e?wsid=/subscriptions/52061d21-01dd-4f9e-aca9-60fff4d67ee2/resourcegroups/MLOpsWorkshop/workspaces/mlops&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
PipelineRun Status: Running


StepRunId: ac02198e-ff79-44b4-8d60-65627366c535
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/ac02198e-ff79-44b4-8d60-65627366c535?wsid=/subscriptions/52061d21-01dd-4f9e-aca9-60fff4d67ee2/resourcegroups/MLOpsWorkshop/workspaces/mlops&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
StepRun( Train Model ) Status: Running

Streaming azureml-logs/55_azureml-execution-tvmps_b17f9ce8e6aea49a4858c02313a900d001f357c8d41296468f05cba4e54bfe58_d.txt
2021-05-19T20:45:41Z Successfully mounted a/an Blobfuse File System at /mnt/batch/tasks/shared/LS_root/jobs/mlops/azureml/ac02198e-ff79-44b4-8d60-65627366c535/mounts/workspaceblobstore
2021-05-19T20:45:42Z Starting ou

'Finished'

In [63]:
# Print the model name, version, tag, and properties
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

driver_model version: 3
	 Training context : Pipeline


driver_model.pkl version: 4


driver_model.pkl version: 3


driver_model version: 2
	 Training context : Pipeline


driver_model.pkl version: 2


driver_model version: 1
	 Training context : Pipeline


driver_model.pkl version: 1


compliance-classifier version: 17
	 type : classification
	 run_id : 0767a613-e8c2-4811-ae6b-64369e340725
	 build_number : 20201217.1


BikeBuyer.mml version: 4


AutoMLb9be0a22f28 version: 1


compliance-classifier version: 16
	 type : classification
	 run_id : a7eb32c9-3207-4a2c-865a-d65fba7946ba
	 build_number : 20201119.1


compliance-classifier version: 15
	 type : classification
	 run_id : 8e84d5f5-54c9-48c9-b733-9bbbcd86558b
	 build_number : 20201118.1


IBM_attrition_model version: 1
	 area : HR
	 type : attrition


AutoML015fd913221 version: 1


diabetes_model version: 1
	 Training context : Inline Training
	 AUC : 0.8857431111811085
	 Accuracy : 0.9002222222222223


glove-text-classifier versi